# CAPTSTONE PROJECT 3:

### DATA CLEANING

***

#### CLEAN DATA:
* Import .csv file in local Data Folder as DataFrame: df
* Examine the contents of DataFrame and address any issues regard NaN or others empty values
* Save updated dataFrame as df_cleaned.csv

In [8]:
# Import necessary modules for DataFrame cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import finnhub
import nasdaqdatalink
from polygon import RESTClient
from dotenv import load_dotenv
import requests as r
import json
import collections as coll

##### PULL DATA FROM API's and Import public table from S & P 500
***

In [11]:
load_dotenv(".env")
NASDAQ_DATA_LINK_API_KEY = os.getenv('NASDAQ_DATA_LINK_API_KEY')
MKTS_API_KEY = os.getenv('MARKETSTACK_API_KEY')
FNH_API_KEY = os.getenv('FINNHUB_API_KEY')
PLY_API_KEY = os.getenv('POLYGON_API_KEY')

# print(f"NASDAQ KEY: {NASDAQ_DATA_LINK_API_KEY}")
# print(f"MARKETSTACK KEY: {MKTS_API_KEY}")
# print(f"FINNHUB KEY: {FNH_API_KEY}")
# print(f"POLYGON KEY: {PLY_API_KEY}")

NASDAQ KEY: vCoBDwKmDFdadaYVMcoY
MARKETSTACK KEY: a9c064b95f02774025539939980a0e41
FINNHUB KEY: co5fa69r01qrjq1g0bp0co5fa69r01qrjq1g0bpg
POLYGON KEY: yQ2lQqUwhSnsCSvrNKpP7B2Z1hB3qX3b


In [14]:
# Pull data from NASDAQ
nasdaqdatalink.read_key('.env')
